## Import files

In [ ]:
import os
import numpy as np
import pandas as pd

sub_path = "/content/drive/My Drive/Colab Notebooks/ML stacking/best_prediction"
all_files = [name for name in os.listdir(sub_path)]
print(all_files)

# Read and concatenate submissions
outs = [pd.read_csv(os.path.join(sub_path, f), index_col=0) for f in all_files]
concat_sub = pd.concat(outs, axis=1)
cols = list(map(lambda x: "is_iceberg_" + str(x), range(len(concat_sub.columns))))
concat_sub.columns = cols
concat_sub.reset_index(inplace=True)
concat_sub.head()

['200_ens_densenet.csv', 'Keras+TF.csv', 'ResNeXt.csv', 'submission38.csv', 'submission43.csv']


,id,is_iceberg_0,is_iceberg_1,is_iceberg_2,is_iceberg_3,is_iceberg_4
0,5941774d,1.943216e-02,0.005586,1.980650e-02,0.008875,0.005586
1,4023181e,3.168809e-02,0.145927,8.600441e-02,0.146779,0.031688
2,b20200e4,4.000000e-08,0.000015,3.875543e-17,0.048646,0.001000
3,e7f018bb,9.925741e-01,0.999914,9.778560e-01,0.999184,0.999000
4,4371c8c3,2.215107e-02,0.033843,2.992679e-03,0.277251,0.022151


In [ ]:
def prepare_submission_train(proba, path):
    test = pd.read_json("/content/drive/My Drive/Colab Notebooks/ML stacking/data/test.json")
    proba = np.array(proba)

    submission = pd.DataFrame()
    submission['id'] = test['id']
    submission['is_iceberg'] = proba.reshape((-1, 1))
    submission.to_csv(path, float_format='%.15f', index=False)

In [ ]:
# cutoff threshold for lower and upper bounds
cutoff_lo = 0.90
cutoff_hi = 0.10

# get the data fields ready for stacking
concat_sub['is_iceberg_max'] = concat_sub.iloc[:, 1:6].max(axis=1)
concat_sub['is_iceberg_min'] = concat_sub.iloc[:, 1:6].min(axis=1)
concat_sub['is_iceberg_mean'] = concat_sub.iloc[:, 1:6].mean(axis=1)
concat_sub['is_iceberg_median'] = concat_sub.iloc[:, 1:6].median(axis=1)

## MinMax

In [ ]:
x = concat_sub['is_iceberg_0']
y = concat_sub['is_iceberg_1']
concat_sub['is_iceberg'] = np.where((x > 0.5) & (y > 0.5), np.maximum(x, y), np.where((x < 0.5) & (y < 0.5), np.minimum(x, y), x))
prepare_submission_train(concat_sub['is_iceberg'].to_numpy(), path='./Stacked_MinMax.csv')
concat_sub.head()

,id,is_iceberg_0,is_iceberg_1,is_iceberg_2,is_iceberg_3,is_iceberg_4,is_iceberg_max,is_iceberg_min,is_iceberg_mean,is_iceberg_median,is_iceberg
0,5941774d,1.943216e-02,0.005586,1.980650e-02,0.008875,0.005586,0.019807,5.586000e-03,0.011857,0.008875,5.586202e-03
1,4023181e,3.168809e-02,0.145927,8.600441e-02,0.146779,0.031688,0.146779,3.168800e-02,0.088417,0.086004,3.168809e-02
2,b20200e4,4.000000e-08,0.000015,3.875543e-17,0.048646,0.001000,0.048646,3.875543e-17,0.009932,0.000015,4.000000e-08
3,e7f018bb,9.925741e-01,0.999914,9.778560e-01,0.999184,0.999000,0.999914,9.778560e-01,0.993706,0.999000,9.999136e-01
4,4371c8c3,2.215107e-02,0.033843,2.992679e-03,0.277251,0.022151,0.277251,2.992679e-03,0.071678,0.022151,2.215107e-02


## MinMax + BestBase

In [ ]:
# load the model with best base performance
sub_base = pd.read_csv('/content/drive/My Drive/Colab Notebooks/ML stacking/best_prediction/ResNeXt.csv')
concat_sub['is_iceberg_base'] = sub_base['is_iceberg']
concat_sub['is_iceberg'] = np.where(np.all(concat_sub.iloc[:,1:4] > cutoff_lo, axis=1), 
                                    concat_sub['is_iceberg_max'], 
                                    np.where(np.all(concat_sub.iloc[:,1:4] < cutoff_hi, axis=1),
                                             concat_sub['is_iceberg_min'], 
                                             concat_sub['is_iceberg_base']))
concat_sub['is_iceberg'] = np.clip(concat_sub['is_iceberg'].values, 0.001, 0.999)
prepare_submission_train(concat_sub['is_iceberg'].to_numpy(), path='./Stacked_MinMax_BestBase.csv')
concat_sub.head()

,id,is_iceberg_0,is_iceberg_1,is_iceberg_2,is_iceberg_3,is_iceberg_4,is_iceberg_max,is_iceberg_min,is_iceberg_mean,is_iceberg_median,is_iceberg,is_iceberg_base
0,5941774d,1.943216e-02,0.005586,1.980650e-02,0.008875,0.005586,0.019807,5.586000e-03,0.011857,0.008875,0.005586,1.980650e-02
1,4023181e,3.168809e-02,0.145927,8.600441e-02,0.146779,0.031688,0.146779,3.168800e-02,0.088417,0.086004,0.086004,8.600441e-02
2,b20200e4,4.000000e-08,0.000015,3.875543e-17,0.048646,0.001000,0.048646,3.875543e-17,0.009932,0.000015,0.001000,3.875543e-17
3,e7f018bb,9.925741e-01,0.999914,9.778560e-01,0.999184,0.999000,0.999914,9.778560e-01,0.993706,0.999000,0.999000,9.778560e-01
4,4371c8c3,2.215107e-02,0.033843,2.992679e-03,0.277251,0.022151,0.277251,2.992679e-03,0.071678,0.022151,0.002993,2.992679e-03


## MinMax + Median

In [ ]:
concat_sub['is_iceberg'] = np.where(np.all(concat_sub.iloc[:,1:6] > cutoff_lo, axis=1), 
                                   concat_sub['is_iceberg_max'], 
                                   np.where(np.all(concat_sub.iloc[:,1:6] < cutoff_hi, axis=1),
                                            concat_sub['is_iceberg_min'], 
                                            concat_sub['is_iceberg_median']))
concat_sub['is_iceberg'] = np.clip(concat_sub['is_iceberg'].values, 0.001, 0.999)
prepare_submission_train(concat_sub['is_iceberg'].to_numpy(), path='./Stacked_MinMax_Median.csv')
concat_sub.head()

,id,is_iceberg_0,is_iceberg_1,is_iceberg_2,is_iceberg_3,is_iceberg_4,is_iceberg_max,is_iceberg_min,is_iceberg_mean,is_iceberg_median,is_iceberg,is_iceberg_base
0,5941774d,1.943216e-02,0.005586,1.980650e-02,0.008875,0.005586,0.019807,5.586000e-03,0.011857,0.008875,0.005586,1.980650e-02
1,4023181e,3.168809e-02,0.145927,8.600441e-02,0.146779,0.031688,0.146779,3.168800e-02,0.088417,0.086004,0.086004,8.600441e-02
2,b20200e4,4.000000e-08,0.000015,3.875543e-17,0.048646,0.001000,0.048646,3.875543e-17,0.009932,0.000015,0.001000,3.875543e-17
3,e7f018bb,9.925741e-01,0.999914,9.778560e-01,0.999184,0.999000,0.999914,9.778560e-01,0.993706,0.999000,0.999000,9.778560e-01
4,4371c8c3,2.215107e-02,0.033843,2.992679e-03,0.277251,0.022151,0.277251,2.992679e-03,0.071678,0.022151,0.022151,2.992679e-03


## MinMax + Mean

In [ ]:
concat_sub['is_iceberg'] = np.where(np.all(concat_sub.iloc[:,1:6] > cutoff_lo, axis=1), 
                                   concat_sub['is_iceberg_max'], 
                                   np.where(np.all(concat_sub.iloc[:,1:6] < cutoff_hi, axis=1),
                                            concat_sub['is_iceberg_min'], 
                                            concat_sub['is_iceberg_mean']))
prepare_submission_train(concat_sub['is_iceberg'].to_numpy(), path='./Stacked_MinMax_Mean.csv')
concat_sub.head()

,id,is_iceberg_0,is_iceberg_1,is_iceberg_2,is_iceberg_3,is_iceberg_4,is_iceberg_max,is_iceberg_min,is_iceberg_mean,is_iceberg_median,is_iceberg,is_iceberg_base
0,5941774d,1.943216e-02,0.005586,1.980650e-02,0.008875,0.005586,0.019807,5.586000e-03,0.011857,0.008875,5.586000e-03,1.980650e-02
1,4023181e,3.168809e-02,0.145927,8.600441e-02,0.146779,0.031688,0.146779,3.168800e-02,0.088417,0.086004,8.841726e-02,8.600441e-02
2,b20200e4,4.000000e-08,0.000015,3.875543e-17,0.048646,0.001000,0.048646,3.875543e-17,0.009932,0.000015,3.875543e-17,3.875543e-17
3,e7f018bb,9.925741e-01,0.999914,9.778560e-01,0.999184,0.999000,0.999914,9.778560e-01,0.993706,0.999000,9.999136e-01,9.778560e-01
4,4371c8c3,2.215107e-02,0.033843,2.992679e-03,0.277251,0.022151,0.277251,2.992679e-03,0.071678,0.022151,7.167767e-02,2.992679e-03
